In [1]:
import Pkg; Pkg.activate(@__DIR__);
Pkg.add("ControlSystems");
Pkg.add("OSQP");
Pkg.add("Plots");
Pkg.add("SparseArrays");
Pkg.add("LinearAlgebra");
Pkg.add("ForwardDiff");
Pkg.add("MeshCat");
Pkg.add("RobotZoo");
Pkg.add("CoordinateTransformations");
Pkg.add("Rotations");
Pkg.add("Colors");
Pkg.add("StaticArrays");
Pkg.add("RobotDynamics");
Pkg.add("GeometryBasics");
Pkg.add("Test");
Pkg.add("Metaheuristics");
Pkg.instantiate()

  Activating project at `/notebooks/notebooks`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `/notebooks/notebooks/Project.toml`
  No Changes to `/notebooks/notebooks/Ma

In [2]:
# Model parameters
using LinearAlgebra
using Plots
using SparseArrays
using ForwardDiff
using ControlSystems
using OSQP
using Test

In [3]:

"""
 Test -The model of the quadrotor
"""
mutable struct Model
    g::Float64 # gravity
    m::Float64 # mass
    ℓ::Float64 # inter-propeller distance
    J::Float64 # inertia momment
    R::Float64 # propeller radius
    prop_min_h::Float64 # TODO
    ρ::Float64 # GE correction factor (https://doi.org/10.1109/ChiCC.2015.7260521)
    umin::Vector{Float64} # Thrust inferior limit
    umax::Vector{Float64} # Thrust superior limit
end

"""
Todo..
"""
struct Simulation
    h_universe::Float64      # Universe Time Step
    h_controller::Float64    # Controler Time Step
    Tfinal::Float64          # Final simulation time (it always start from time 0)
    Nt_universe::Int64       # Universe Number of time-steps
    Nt_controller::Int64     # Controller Number of time-steps

    function Simulation(h_universe, h_controller, Tfinal)
        Nt_universe = Int(Tfinal/h_universe) + 1    # universe number of time steps
        Nt_controller = Int(Tfinal/h_controller) + 1    # controller number of time steps

        new(h_universe, h_controller, Tfinal, Nt_universe, Nt_controller)
    end
end
"""
    Defines the trajectory parameters, the trajectory for now is a sine
"""
mutable struct Trajectory
    Amp::Float64    # Sine amplitude - [meters]
    freq::Float64   # Sine frequency - [Hz]
    init_heigth::Float64 # Initial height [m]
    stat_height::Float64 # Stationary height [m]
end

function ground_effect(height, prop_radius, adjust_factor, height_saturation)
    saturated_height = max(height, height_saturation)
    return (1/(1 - adjust_factor*(prop_radius/(4*saturated_height))^2))
end

"""
    TODO...
"""
function traj_y(traj_params, t)
    local A = traj_params.Amp
    local f = traj_params.freq
    local offset = traj_params.stat_height

    return A * sin(2*pi*f*t) + offset
end
"""
    TODO...
"""
function traj_y_dot(traj_params, t)
    local A = traj_params.Amp
    local f = traj_params.freq

    return (A*2*pi*f) * cos(2*pi*f*t)
end

"""
    h_c - time step (controller)
"""

function gen_ref(state_space, traj_params, tunning_params, t, h_c)
    local Nh = tunning_params.Nh
    local Nx = state_space.Nx
    x_ref = zeros(Nx * Nh) # Initialize the reference with zeros

    for j = 1:Nh # for each element on the predictive horizon
        x_ref[Nx*(j-1) + 2] = traj_y(traj_params, t+(j*h_c))
        x_ref[Nx*(j-1) + 5] = traj_y_dot(traj_params, (t+(j*h_c)))
    end

    return x_ref
end

# Planar Quadrotor Dynamics for modeling
"""
    TODO...
"""
function quad_dynamics(model, x, u, plat_h)
    θ = x[3]
    GE = ground_effect(x[2] - plat_h, model.R, model.ρ, model.prop_min_h)

    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)*GE

    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ)*GE - model.g

    ô= (1/model.J)*(model.ℓ/2)*(u[2] - u[1])

    return [x[4:6]; ẍ; ÿ; ô]
end

"""
    TODO...
"""
function dynamics_rk4(x,u, dynamics, sim_param)
    # RK4 integration with zero-order hold on u
    local h_u = sim_param.h_universe

    # VERIFICAR A ALTURA ADEQUADA DA PLATAFORMA (Se é adequada para o RK4)
    f1 = dynamics(x, u)
    f2 = dynamics(x + 0.5 * h_u * f1, u)
    f3 = dynamics(x + 0.5 * h_u * f2, u)
    f4 = dynamics(x + h_u * f3, u)
    return x + (h_u / 6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

function find_hover_conditions(model, height)
   # @test height >= model.prop_min_h
   #
    x_hover = [0.0, max(height, model.prop_min_h), 0.0, 0.0, 0.0, 0.0]
    weight = model.m * model. g
    GE = ground_effect(height, model.R, model.ρ, model.prop_min_h)
    u_hover = [
        0.5* weight / GE;
        0.5* weight / GE
    ]
    return x_hover, u_hover
end

mutable struct MPCTunningParameters
    Nh::Int64           # Predictive horizon
    Nc::Int64           # Control horizon (Nc < Nh)
    Q::Matrix{Float64}  # Stage State Cost Penalty
    R::Matrix{Float64}  # Stage Control Cost Penalty
    Qn::Matrix{Float64} # Terminal State Cost Penalty
end

# Cost function
"""
    MAYBE ERASE THIS FUNCTION
"""
function cost(xhist,uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(size(xhist,2)-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:,k] + 0.5*(uhist[k]'*R*uhist[k])[1]
    end
    return cost
end

"""
    TODO...
"""
struct LinearStateSpace
    Nx::Int64          # State vector dimension
    Nu::Int64          # Control vector dimension
    A::Matrix{Float64} # State Matrix
    B::Matrix{Float64} # Control Matrix
    x_hover::Vector{Float64}   # x value for which the linearization was made
    u_hover::Vector{Float64}   # u value for which the linearization was made

    function LinearStateSpace(model, simulation, Nx, Nu, height)
        x_hover, u_hover = find_hover_conditions(model, height)
        A = ForwardDiff.jacobian(x->
            dynamics_rk4(x, u_hover, (x,u)->quad_dynamics(model, x, u, height), simulation)
            ,x_hover)
        B = ForwardDiff.jacobian(u ->
            dynamics_rk4(x_hover, u, (x,u)->quad_dynamics(model, x, u, height), simulation)
        , u_hover)

        new(Nx, Nu, A, B, x_hover, u_hover)
    end
end

"""
TODO...
"""
mutable struct MPCMatrices
    P::Matrix{Float64}
    U::Matrix{Float64}
    Θ::Matrix{Float64}
    H::SparseMatrixCSC{Float64,Int64}
    b::Vector{Float64}
    C::SparseMatrixCSC{Float64,Int64}
    Y::Matrix{Float64}
    D::SparseMatrixCSC{Float64,Int64}
    lb::Vector{Float64}
    ub::Vector{Float64}

    """
        trajectory is temporarily here because i'm trying to make the problem linear again...
    """
    function MPCMatrices(model, state_space, tunning_params, trajectory)
        A = state_space.A
        B = state_space.B
        Q = tunning_params.Q
        R = tunning_params.R
        Qn = tunning_params.Qn
        Nx = state_space.Nx
        Nu = state_space.Nu
        Nh = tunning_params.Nh
        Nc = tunning_params.Nc
        x_hover = state_space.x_hover
        u_hover = state_space.u_hover

        # Compute P from dare (Direct a... Ricatti Equation)
        P = dare(A, B, Q, R) # daRicattiEquation (Pq isso ta aqui??) STUDY THIS!!!
        
        U = kron(Diagonal(I,Nh), [I zeros(Nu,Nx)]) # Matrix that picks out all u
        Θ = kron(Diagonal(I,Nh), [0 0 0 0 1 0 0 0]) # Matrix that picks out all x3 (θ) ... explain this better TODO!!
        H = sparse([kron(Diagonal(I,Nh-1),[R zeros(Nu,Nx); zeros(Nx,Nu) Q]) zeros((Nx+Nu)*(Nh-1), Nx+Nu); zeros(Nx+Nu,(Nx+Nu)*(Nh-1)) [R zeros(Nu,Nx); zeros(Nx,Nu) P]])
        b = zeros(Nh*(Nx+Nu))
        C = sparse([[B -I zeros(Nx,(Nh-1)*(Nu+Nx))]; zeros(Nx*(Nh-1),Nu) [kron(Diagonal(I,Nh-1), [A B]) zeros((Nh-1)*Nx,Nx)] + [zeros((Nh-1)*Nx,Nx) kron(Diagonal(I,Nh-1),[zeros(Nx,Nu) Diagonal(-I,Nx)])]])
        # Cria uma matriz que seleciona o 4º elemento do bloco (ou seja, o y)
        Y = kron(Diagonal(I, Nh), [0 0 0 1 0 0 0 0])
        D = [C; U; Θ; Y]
        #local x_hover, u_hover = find_hover_conditions(model, temp_height) # Solve this problem TODO!

        lb = [zeros(Nx*Nh);
              kron(ones(Nh), model.umin - u_hover);
              -0.2 * ones(Nh);
              -4.0 * ones(Nh)]
        ub = [zeros(Nx*Nh);
              kron(ones(Nh), model.umax - u_hover);
              0.2 * ones(Nh);
              4.0 * ones(Nh)]
        new(P, U, Θ, H, b, C, Y, D, lb, ub)
    end
end

# MPC Controller
function mpc_controller(model, simulation, tunning_params, trajectory, t, x, xref)
    Q = tunning_params.Q
    Nh = tunning_params.Nh
    Nu = 2
    Nx = 6
    #height = x[2] - (xref[2] -  trajectory.stat_height)
    #lstate_space = LinearStateSpace(model, simulation, Nx, Nu, height)

    lstate_space = LinearStateSpace(model, simulation, Nx, Nu, trajectory.stat_height)

    A_prime = lstate_space.A
    B_prime = lstate_space.B
    x_hover = lstate_space.x_hover
    u_hover = lstate_space.u_hover

    # ....
    mpc_mats = MPCMatrices(model, lstate_space, tunning_params, trajectory)

    P = mpc_mats.P
    H = mpc_mats.H
    b = mpc_mats.b
    D = mpc_mats.D
    lb = mpc_mats.lb
    ub = mpc_mats.ub

    #Update QP problem
    lb[1:6] .= -A_prime * x
    ub[1:6] .= -A_prime * x

    # Atualiza os limites para a restrição Y
    # Índices para Y: (Nx*Nh + 2*Nh + 1) até (Nx*Nh + 3*Nh)

    #y_indices = (Nx*Nh + 3*Nh + 1):(Nx*Nh + 4*Nh)
    #lb[y_indices] .= xref[Nx + 2]

    #println(xref[Nh*Nx-4])
    # Efetua \"rastreamento de referência
    # Cte se xref não mudar
    
    for j = 1:(Nh-1)
        b[(Nu+(j-1)*(Nx+Nu)).+(1:Nx)] .= -Q*xref[1 + Nx*(j-1):Nx* j]
        #b[(Nu+(j-1)*(Nx+Nu)).+(1:Nx)] .= -Q*xref[1:Nx]
    end
    b[(Nu+(Nh-1)*(Nx+Nu)).+(1:Nx)] .= -P*xref[(Nh*Nx-Nx)+1:Nx*Nh]
    #b[(Nu+(Nh-1)*(Nx+Nu)).+(1:Nx)] .= -P*xref[1:Nx]
    #OSQP.update!(prob, q=b, l=lb, u=ub)
    prob = OSQP.Model()
    OSQP.setup!(prob; P=H, q=b, A=D, l=lb, u=ub, verbose=false, eps_abs=1e-8, eps_rel=1e-8, polish=1);

    #Solve QP
    results = OSQP.solve!(prob)
    Δu = results.x[1:Nu]
    #println(results.x)

    return u_hover + Δu
end

function verify_landing(x, x_ref)
    #crit = [0.05; 0.05; 0.05; 0.10; 0.02; 0.1]
    crit = [0.05; 0.05; 0.05; 0.05; 0.05; 0.05]

    sucess_check = true
    for i = 1:(Nx)
        if (abs(x[i] - x_ref[i]) >= crit[i])
             sucess_check = false
        end
    end
    return ~sucess_check
end

function closed_loop(x0, state_space, tunning_params, traj_params, simulation, model, controller)
    local h_c = simulation.h_controller
    local h_uni = simulation.h_universe

    local N_c = simulation.Nt_controller
    local N_uni = simulation.Nt_universe

    local umax = model.umax
    local umin = model.umin

    xhist = zeros(length(x0), N_uni)

    u0 = controller(1, x0)

    uhist = zeros(length(u0), N_uni - 1)
    plathist = zeros(N_uni)
    drefyhist = zeros(N_uni)
    refyhist = zeros(N_uni)

    uhist[:,1] .= u0
    xhist[:,1] .= x0

    local not_landed = true
    land_time = 0.0

    uk = u0
    N_ratio = Int(round(h_c / h_uni)) ## h_c need to be multiple of h_u

    for k = 1:(N_uni-1)
        local curr_time_inst = k * h_uni

        if(k % N_ratio == 0)
            uk = controller(curr_time_inst, xhist[:, k])
        end

        if not_landed
            not_landed = verify_landing(xhist[:, k], gen_ref(state_space, traj_params, tunning_params, curr_time_inst, h_uni))
            land_time = curr_time_inst
        end

        foo = gen_ref(state_space, traj_params, tunning_params, curr_time_inst, h_uni)
        refyhist[k] = foo[2]
        drefyhist[k] = foo[5]

        plathist[k] = traj_y(traj_params, curr_time_inst) - traj_params.stat_height # Recover the current platform height
        if k == N_uni # Goes to the N_uni - 1 step
            continue
        end

        uhist[:, k] = max.(min.(umax, uk), umin) #enforce control limits

        local height = xhist[2, k] - plathist[k]
        xhist[:, k + 1] .= dynamics_rk4(xhist[:, k],uhist[:, k],(x,u)->quad_dynamics(model, x, u, height), simulation)
    end

    return xhist, uhist, plathist, refyhist, drefyhist, land_time
end


closed_loop (generic function with 1 method)

In [ ]:
## Model Configs
#    g::Float64 # gravity
#    m::Float64 # mass
#    ℓ::Float64 # inter-propeller distance
#    J::Float64 # inertia momment
#    R::Float64 # propeller radius
#    prop_min_h::Float64 #
#    ρ::Float64 # GE correction factor (https://doi.org/10.1109/ChiCC.2015.7260521)
#    umin::Vector{Float64} # Thrust inferior limit
#    umax::Vector{Float64} # Thrust superior limit

model = Model(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [0.0, 0.0], [0.0, 0.0])
model.g = 9.81 # [m/s^2]
model.m = 1.0 # [kg]
model.ℓ = 0.3 # [meters]
model.J = 0.2 * model.m * model.ℓ * model.ℓ # [kg . m^2]
model.R = 0.15 # [meters]
model.prop_min_h = 0.1 # [meters]
model.ρ = 1.0 # dimensionless

#Thrust limits
model.umin = [0.0 * model.m * model.g; 0.0 * model.m * model.g]
model.umax = [2.0 * model.m * model.g; 2.0 * model.m * model.g]

## Simulation Configs
#
# Universe 1000Hz
# Controlador 100Hz

h_universe = 0.0001 # Time step (10 kHz)
h_controller = 0.004 # Time step (200 Hz)
Tfinal = 10.0 # Simulate from 0 - 10 seconds

simulation = Simulation(h_universe, h_controller, Tfinal)

## Trajectory configuration
#mutable struct Trajectory
#    Amp::Float64    # Sine amplitude - [meters]
#    freq::Float64   # Sine frequency - [Hz]
#end

landing_start_height = 3.0
stationary_height = model.prop_min_h + 0.06 # 6 centimeters above the ground
plat_amplitude = 0.1
plat_freq = 0.3

traj_params = Trajectory(plat_amplitude, plat_freq, landing_start_height, stationary_height)

## Linear State Space Configs ???
#struct LinearStateSpace
#    Nx::Int64          # State vector dimension
#    Nu::Int64          # Control vector dimension
#    A::Matrix{Float64} # State Matrix
#    B::Matrix{Float64} # Control Matrix
#    x_hover::Float64   # x value for which the linearization was made
#    u_hover::Float64   # u value for which the linearization was made
Nx = 6
Nu = 2

#    function LinearStateSpace(model, simulation, Nx, Nu, height)
state_space = LinearStateSpace(model, simulation, Nx, Nu, landing_start_height)

## Tunning Parameters Configs
# mutable struct MPCTunningParameters
#    Nh::Int64           # Predictive horizon
#    Nc::Int64           # Control horizon (Nc < Nh)
#    Q::Matrix{Float64}  # Stage State Cost Penalty
#    R::Matrix{Float64}  # Stage Control Cost Penalty
#    Qn::Matrix{Float64} # Terminal State Cost Penalty
#end
Nh = 10
Nc = 42 # For now is unused...

# Cost weights
Q = Array(10.0*I(Nx));
Q[2,2] = 1000.0
R = Array(.01*I(Nu));
Qn = Array(1000.0*I(Nx));

tunning_params = MPCTunningParameters(Nh, Nc, Q, R, Qn)

## MPC Matrices Configs

#mutable struct MPCMatrices
#    P::Matrix{Float64}
#    U::Matrix{Float64}
#    Θ::Matrix{Float64}
#    H::SparseMatrixCSC{Float64,Int64}
#    b::Array{Float64,1}
#    C::SparseMatrixCSC{Float64,Int64}
#    Y::Matrix{Float64}
#    D::Array{Float64,1}
#    lb::Array{Float64,1}
#    ub::Array{Float64,1}

# function MPCMatrices(model, state_space, tunning_params, trajectory)
#mpc_mats =  MPCMatrices(model, state_space, tunning_params, traj_params)"

#x_ref = [0.0; 1.0; 0; 0; 0; 0]
x0 = [1.0; traj_params.init_heigth; 0.0; 0; 0; 0]

#xhist1, uhist1 = closed_loop(x0, (t,x)->lqr_controller(t,x,K,x_ref), Nt);
#xhist2, uhist2 = closed_loop(x0, (t,x)->mpc_controller(t,x,x_ref), Nt);
xhist3, uhist3, plathist3, refyhist3, drefyhist3, land_time = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,
    # mpc_controller(model, simulation, tunning_params, trajectory, t, x, xref)
    (t, x) -> mpc_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        t,
        x, 
        gen_ref(state_space, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    )
);

In [ ]:
thist = Array(range(0, simulation.h_universe * (simulation.Nt_universe - 1), step = simulation.h_universe));
#println(simulation.h_universe * (simulation.Nt_universe - 1))
plot(thist, xhist3[1,:], label="x MPC")

scatter!([land_time], [xhist3[1,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)
xlabel!("time")

In [ ]:
plot(thist, xhist3[4, :], label="x dot MPC")
scatter!([land_time], [xhist3[4,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

xlabel!("time")

In [ ]:
#plot(thist,xhist1[2,:], label=\"y LQR\")
plot(thist,xhist3[3,:], label="θ MPC")
scatter!([land_time], [xhist3[3,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)
xlabel!("time")

In [ ]:
plot(thist, xhist3[6, :], label="θ dot MPC")
scatter!([land_time], [xhist3[6,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

xlabel!("time")

In [ ]:
## Plot de interesse|
plot(thist,xhist3[2,:], label="y MPC")

#plot!(thist, plathist3 .+ traj_params.stat_height, label=\"traj\")
plot!(thist, refyhist3, label="traj")
plot!(thist, plathist3, label="plat")

In [ ]:
xlabel!("time")
scatter!([land_time], [xhist3[2,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

In [ ]:
plot(thist,xhist3[5,:], label="y MPC")
plot!(thist, drefyhist3, label="cos")

In [ ]:
xlabel!("time")
scatter!([land_time], [drefyhist3[trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

In [ ]:
plot(thist[1:end-1], uhist3[1,:], label="u1 MPC")
xlabel!("Time")

In [ ]:
plot(thist[1:end-1], uhist3[2,:], label="u2 MPC")
xlabel!("Time")